In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fraudfinder - ML Pipeline

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/vertex_ai/06_formalization.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/vertex_ai/06_formalization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/vertex_ai/06_formalization.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

This notebook shows how to use Feature Store, Pipelines and Model Monitoring for building an end-to-end demo using both components defined in `google_cloud_pipeline_components` and custom components. 

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)

Steps performed in this notebook:

* Create a Vetex AI Pipeline to orchestrate and automate the ML workflow

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)

### Import libraries and define constants

#### Libraries
Next you will import the libraries needed for this notebook. 

Note that currently this notebook uses KFP SDK v1, whereas the environment includes KFP v2. As an interim solution, we will downlevel KFP and the Google Cloud Pipeline Components in order to use the v1 code here as-is. See the [KFP migration guide](https://www.kubeflow.org/docs/components/pipelines/v2/migration/) for more details of moving from v1 to v2. 

In [ ]:
# General
import os
import sys
import random
from datetime import datetime, timedelta
import json

# Vertex Pipelines
from typing import NamedTuple
import kfp
from kfp import dsl
from kfp.dsl import (
    Artifact,
    Input,
    Model,
    Output,
    OutputPath,
    Metrics,
    ClassificationMetrics,
    component,
)
from kfp import compiler

from google.cloud import aiplatform as vertex_ai

In [ ]:
from google_cloud_pipeline_components.v1.dataset import TabularDatasetCreateOp

from google_cloud_pipeline_components.v1.custom_job import CustomTrainingJobOp
from google_cloud_pipeline_components.v1.endpoint import (
    EndpointCreateOp,
    ModelDeployOp,
)
from google_cloud_pipeline_components.v1.model import ModelUploadOp

from google_cloud_pipeline_components.types import artifact_types

In [ ]:
print("kfp version:", kfp.__version__)

#### Variables

In [ ]:
# Components variables
BASE_IMAGE = "python:3.7"
COMPONENTS_DIR = os.path.join(os.curdir, "pipelines", "components")
INGEST_FEATURE_STORE = f"{COMPONENTS_DIR}/ingest_feature_store_{ID}.yaml"
EVALUATE = f"{COMPONENTS_DIR}/evaluate_{ID}.yaml"

# Pipeline variables
PIPELINE_NAME = f"fraud-finder-xgb-pipeline-{ID}"
PIPELINE_DIR = os.path.join(os.curdir, "pipelines")
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipelines"
PIPELINE_PACKAGE_PATH = f"{PIPELINE_DIR}/pipeline_{ID}.json"

# Feature Store component variables
BQ_DATASET = "tx"
READ_INSTANCES_TABLE = f"ground_truth_{ID}"
READ_INSTANCES_URI = f"bq://{PROJECT_ID}.{BQ_DATASET}.{READ_INSTANCES_TABLE}"

# Dataset component variables
DATASET_NAME = f"fraud_finder_dataset_{ID}"

# Training component variables
JOB_NAME = f"fraudfinder-train-xgb-{ID}"
MODEL_NAME = f"{MODEL_NAME}_xgb_pipeline_{ID}"
CONTAINER_URI = "us-docker.pkg.dev/vertex-ai/training/xgboost-cpu.1-1:latest"
MODEL_SERVING_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.2-0:latest"
)
ARGS = ["--bucket", f"gs://{BUCKET_NAME}"]
IMAGE_REPOSITORY = f"fraudfinder-{ID}"
IMAGE_NAME = "dask-xgb-classificator"
IMAGE_TAG = "v1"
IMAGE_URI = f"us-central1-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REPOSITORY}/{IMAGE_NAME}:{IMAGE_TAG}"  # TODO: get it from config

# Evaluation component variables
METRICS_URI = f"gs://{BUCKET_NAME}/deliverables/metrics.json"
AVG_PR_THRESHOLD = 0.2
AVG_PR_CONDITION = "avg_pr_condition"

# Endpoint variables
ENDPOINT_NAME = f"{ENDPOINT_NAME}_xgb_pipeline_{ID}"

#### Initialize the Vertex AI SDK
Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
# Vertex AI SDK
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

In [ ]:
!gsutil ubla set on gs://{BUCKET_NAME}

#### Create directories 
Create a directory for you pipeline and pipeline components. 

In [ ]:
!mkdir -p -m 777 $PIPELINE_DIR $COMPONENTS_DIR

## Build model training file

In [ ]:
%%writefile build_training/train_xgb.py

"""
train_xgb.py is the module for training a XGBClassifier pipeline
"""

# Libraries --------------------------------------------------------------------------------------------------------------------------

import argparse
import numpy as np
import os
import subprocess
import sys
import json
import logging
from pathlib import Path
import dask.dataframe as dask_df
from dask.distributed import LocalCluster, Client
import xgboost as xgb
from sklearn.metrics import (roc_curve, confusion_matrix, average_precision_score, f1_score, 
                            log_loss, precision_score, recall_score)

# Variables --------------------------------------------------------------------------------------------------------------------------

## Read environmental variables
def gcs_path_to_local_path(old_path):
    new_path = old_path.replace("gs://", "/gcs/")
    return new_path

# TRAINING_DATA_PATH = gcs_path_to_local_path(os.environ["AIP_TRAINING_DATA_URI"])
# TEST_DATA_PATH = gcs_path_to_local_path(os.environ["AIP_TEST_DATA_URI"])
# MODEL_DIR = gcs_path_to_local_path(os.environ["AIP_MODEL_DIR"])
# MODEL_PATH = MODEL_DIR + "model.bst"

## Training variables
LABEL_COLUMN = "tx_fraud"
UNUSED_COLUMNS = ["timestamp","entity_type_customer","entity_type_terminal"]
DATA_SCHEMA = {
"timestamp" : "object",
"tx_amount": "float64",
"tx_fraud": "Int64",
"entity_type_customer": "Int64",
"customer_id_nb_tx_1day_window": "Int64",
"customer_id_nb_tx_7day_window": "Int64",
"customer_id_nb_tx_14day_window": "Int64",
"customer_id_avg_amount_1day_window": "float64",
"customer_id_avg_amount_7day_window": "float64",
"customer_id_avg_amount_14day_window": "float64",
"customer_id_nb_tx_15min_window": "Int64",
"customer_id_avg_amount_15min_window": "float64",
"customer_id_nb_tx_30min_window": "Int64",
"customer_id_avg_amount_30min_window": "float64",
"customer_id_nb_tx_60min_window": "Int64",
"customer_id_avg_amount_60min_window": "float64",
"entity_type_terminal": "Int64",
"terminal_id_nb_tx_1day_window": "Int64",
"terminal_id_nb_tx_7day_window": "Int64",
"terminal_id_nb_tx_14day_window": "Int64",
"terminal_id_risk_1day_window": "float64",
"terminal_id_risk_7day_window": "float64",
"terminal_id_risk_14day_window": "float64",
"terminal_id_nb_tx_15min_window": "Int64",
"terminal_id_avg_amount_15min_window": "float64",
"terminal_id_nb_tx_30min_window": "Int64",
"terminal_id_avg_amount_30min_window": "float64",
"terminal_id_nb_tx_60min_window": "Int64",
"terminal_id_avg_amount_60min_window": "float64"
}

# Helpers -----------------------------------------------------------------------------------------------------------------------------
def get_args():
    parser = argparse.ArgumentParser()

    # Data files arguments
    parser.add_argument("--bucket", dest="bucket", type=str,
                        required=True, help="Bucket uri")
    parser.add_argument("--max_depth", dest="max_depth",
                        default=6, type=int,
                        help="max_depth value.")
    parser.add_argument("--eta", dest="eta",
                        default=0.4, type=float,
                        help="eta.")
    parser.add_argument("--gamma", dest="gamma",
                        default=0.0, type=float,
                        help="eta value")
    
    return parser.parse_args()

def resample(df, replace, frac=1, random_state = 8):
    shuffled_df = df.sample(frac=frac, replace=replace, random_state=random_state)
    return shuffled_df

def preprocess(df):
    df = df.drop(columns=UNUSED_COLUMNS)

    # Drop rows with NaN"s
    df = df.dropna()

    # Convert integer valued (numeric) columns to floating point
    numeric_columns = df.select_dtypes(["float32", "float64"]).columns
    numeric_format = {col:"float32" for col in numeric_columns}
    df = df.astype(numeric_format)

    return df

def evaluate_model(model, x_true, y_true):
    y_true = y_true.compute()
    
    #calculate metrics
    metrics={}
    
    y_score =  model.predict_proba(x_true)[:, 1]
    y_score = y_score.compute()
    fpr, tpr, thr = roc_curve(
         y_true=y_true, y_score=y_score, pos_label=True
    )
    fpr_list = fpr.tolist()[::1000]
    tpr_list = tpr.tolist()[::1000]
    thr_list = thr.tolist()[::1000]

    y_pred = model.predict(x_true)
    y_pred.compute()
    c_matrix = confusion_matrix(y_true, y_pred)
    
    avg_precision_score = round(average_precision_score(y_true, y_score), 3)
    f1 = round(f1_score(y_true, y_pred), 3)
    lg_loss = round(log_loss(y_true, y_pred), 3)
    prec_score = round(precision_score(y_true, y_pred), 3)
    rec_score = round(recall_score(y_true, y_pred), 3)
    
    metrics["fpr"] = [round(f, 3) for f in fpr_list]
    metrics["tpr"] = [round(f, 3) for f in tpr_list]
    metrics["thrs"] = [round(f, 3) for f in thr_list]
    metrics["confusion_matrix"] = c_matrix.tolist()
    metrics["avg_precision_score"] = avg_precision_score
    metrics["f1_score"] = f1
    metrics["log_loss"] = lg_loss
    metrics["precision_score"] = prec_score
    metrics["recall_score"] = rec_score
    
    return metrics


def main():
    args = get_args()
    

    # variables
    data_path = f"gs://{args.bucket}/data/snapshots/pipeline_training/000000000000.csv"
    print(data_path)
    # read data
    df = dask_df.read_csv(data_path, dtype=DATA_SCHEMA)
    train_df, test_df = df.random_split([0.8,0.2])
    
    # preprocessing
    preprocessed_train_df = preprocess(train_df)
    preprocessed_test_df = preprocess(test_df)
    
    # downsampling
    train_nfraud_df = preprocessed_train_df[preprocessed_train_df[LABEL_COLUMN]==0]
    train_fraud_df = preprocessed_train_df[preprocessed_train_df[LABEL_COLUMN]==1]
    train_nfraud_downsample = resample(train_nfraud_df,
                          replace=True, 
                          frac=len(train_fraud_df)/len(train_df))
    ds_preprocessed_train_df = dask_df.concat([train_nfraud_downsample, train_fraud_df])
    
    # target, features split
    x_train = ds_preprocessed_train_df[ds_preprocessed_train_df.columns.difference([LABEL_COLUMN])]
    y_train = ds_preprocessed_train_df.loc[:, LABEL_COLUMN].astype(int)
    x_true = preprocessed_test_df[preprocessed_test_df.columns.difference([LABEL_COLUMN])]
    y_true = preprocessed_test_df.loc[:, LABEL_COLUMN].astype(int)
    
    # train model
    cluster =  LocalCluster()
    client = Client(cluster)
    model = xgb.dask.DaskXGBClassifier(objective="reg:logistic", eval_metric="logloss")
    model.client = client 
    model.fit(x_train, y_train, eval_set=[(x_true, y_true)])

    model.save_model("model.bst")
    subprocess.check_call(
        ["gsutil", "cp", "model.bst", f"{args.bucket}/model/model.bst"], stderr=sys.stdout
    )    
    #generate metrics
    metrics = evaluate_model(model, x_true, y_true)
    with open("metrics.json", "w") as file:
        json.dump(metrics, file, sort_keys = True, indent = 4)
    file.close()
    subprocess.check_call(
        ["gsutil", "cp", "metrics.json", f"{args.bucket}/metrics.json"], stderr=sys.stdout
    )
    
if __name__ == "__main__":
    main()

In [ ]:
%%writefile build_training/Dockerfile
# Specifies base image and tag
FROM us-docker.pkg.dev/vertex-ai/training/xgboost-cpu.2-1:latest
WORKDIR /root

# Installs additional packages
RUN pip install dask distributed pyarrow --upgrade

# Copies the trainer code to the docker image.
COPY ./train_xgb.py /root/train_xgb.py

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python3", "train_xgb.py"]

In [ ]:
# Build and push Docker container
!docker build -t $IMAGE_URI ./build_training/
!docker push $IMAGE_URI

print("Done")

### Create a end-to-end Pipeline and execute it on Vertex AI Pipelines.

We will build a pipeline that you will execute using [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction). Vertex AI Pipelines helps you to automate, monitor, and govern your ML systems by orchestrating your ML workflow in a serverless manner, and storing your workflow's artifacts using Vertex ML Metadata. Authoring ML Pipelines that run on Vertex AI pipelines can be done in two different ways:

* [Tensorflow Extended](https://www.tensorflow.org/tfx/guide)
* [Kubeflow Pipelines SDK](https://kubeflow-pipelines.readthedocs.io/en/1.8.13/)

Based on your preference you can choose between the two options. This notebook will only focus on Kubeflow Pipelines.

If you don't have familiarity in authoring pipelines in Vertex AI Pipelines, we suggest the following resources:
* [Introduction to Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)
* [Build a Pipeline in Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline)

### Define Custom Components for your pipeline

We will use a mix of prebuilt (Google Cloud Pipeline Components) and custom components in this notebook. The difference is:

* Prebuilt components are official [Google Cloud Pipeline Components](https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction)(GCPC). The GCPC Library provides a set of prebuilt components that are production quality, consistent, performant, and easy to use in Vertex AI Pipelines.
* As you will build in the cell below, a data scientist or ML engineer typically authored the custom component. This means you have more control over the component (container) code. In this case, it's a Python-function-based component. You also have the option to build a component yourself by packaging code into a container.

In the following two cells, you will build two custom components:

    *Feature Store component.

    *Evaluation component.

#### Define feature store component

Notice that the component assumes that containes the entities-timestamps "query" is already created.

# Feature Store
Next you will build a custom component using the [KFP SDK](https://kubeflow-pipelines.readthedocs.io/en/1.8.13/). Here you will take a Python function and create a component out of it. This component will take features from the Vertex AI Feature Store and output them on Google Cloud Storage (GCS). 

In [ ]:
@component(
    output_component_file=INGEST_FEATURE_STORE,
    base_image=BASE_IMAGE,
    packages_to_install=["google-cloud-aiplatform==1.21.0"],
)
def ingest_features_gcs(
    project_id: str,
    region: str,
    bucket_name: str,
    feature_store_id: str,
    read_instances_uri: str,
) -> NamedTuple("Outputs", [("snapshot_uri_paths", str),],):
    # Libraries --------------------------------------------------------------------------------------------------------------------------
    from datetime import datetime
    import glob
    import urllib
    import json

    # Feature Store
    from google.cloud.aiplatform import Featurestore, EntityType, Feature

    # Variables --------------------------------------------------------------------------------------------------------------------------
    api_endpoint = region + "-aiplatform.googleapis.com"
    bucket = urllib.parse.urlsplit(bucket_name).netloc
    export_uri = (
        f"{bucket_name}/data/snapshots/pipeline_training"  # format as new gsfuse requires
    )
    export_uri_path = f"/gcs/{bucket}/data/snapshots/pipeline_training"
    customer_entity = "customer"
    terminal_entity = "terminal"
    serving_feature_ids = {customer_entity: ["*"], terminal_entity: ["*"]}

    print(bucket)
    print(export_uri)
    print(export_uri_path)
    print(customer_entity)
    print(terminal_entity)
    print(serving_feature_ids)

    # Main -------------------------------------------------------------------------------------------------------------------------------

    ## Define the feature store resource path
    feature_store_resource_path = (
        f"projects/{project_id}/locations/{region}/featurestores/{feature_store_id}"
    )
    print("Feature Store: \t", feature_store_resource_path)

    ## Run batch job request
    try:
        ff_feature_store = Featurestore(feature_store_resource_path)
        ff_feature_store.batch_serve_to_gcs(
            gcs_destination_output_uri_prefix=export_uri,
            gcs_destination_type="csv",
            serving_feature_ids=serving_feature_ids,
            read_instances_uri=read_instances_uri,
            pass_through_fields=["tx_fraud", "tx_amount"],
        )
    except Exception as error:
        print(error)

    # Store metadata
    snapshot_pattern = f"{export_uri_path}/*.csv"
    snapshot_files = glob.glob(snapshot_pattern)
    snapshot_files_fmt = [p.replace("/gcs/", "gs://") for p in snapshot_files]
    snapshot_files_string = json.dumps(snapshot_files_fmt)

    component_outputs = NamedTuple(
        "Outputs",
        [
            ("snapshot_uri_paths", str),
        ],
    )

    print(snapshot_pattern)
    print(snapshot_files)
    print(snapshot_files_fmt)
    print(snapshot_files_string)

    return component_outputs(snapshot_files_string)

#### Define an evaluate custom component
Next you will build a custom component that will evaluate our XGBoost model. This component will output `avg_precision_score` so that it can be used downstream for validating the model before deployment. 

In [ ]:
@component(output_component_file=EVALUATE)
def evaluate_model(
    model_in: Input[Artifact],
    metrics_uri: str,
    meta_metrics: Output[Metrics],
    graph_metrics: Output[ClassificationMetrics],
    model_out: Output[Artifact],
) -> NamedTuple("Outputs", [("metrics_thr", float),],):
    # Libraries --------------------------------------------------------------------------------------------------------------------------
    import json

    # Variables --------------------------------------------------------------------------------------------------------------------------
    metrics_path = metrics_uri.replace("gs://", "/gcs/")
    labels = ["not fraud", "fraud"]

    # Main -------------------------------------------------------------------------------------------------------------------------------
    with open(metrics_path, mode="r") as json_file:
        metrics = json.load(json_file)

    ## metrics
    fpr = metrics["fpr"]
    tpr = metrics["tpr"]
    thrs = metrics["thrs"]
    c_matrix = metrics["confusion_matrix"]
    avg_precision_score = metrics["avg_precision_score"]
    f1 = metrics["f1_score"]
    lg_loss = metrics["log_loss"]
    prec_score = metrics["precision_score"]
    rec_score = metrics["recall_score"]

    meta_metrics.log_metric("avg_precision_score", avg_precision_score)
    meta_metrics.log_metric("f1_score", f1)
    meta_metrics.log_metric("log_loss", lg_loss)
    meta_metrics.log_metric("precision_score", prec_score)
    meta_metrics.log_metric("recall_score", rec_score)
    graph_metrics.log_roc_curve(fpr[1:], tpr[1:], thrs[1:])
    graph_metrics.log_confusion_matrix(labels, c_matrix)

    ## model metadata
    model_framework = "xgb.dask"
    model_type = "DaskXGBClassifier"
    model_user = "author"
    model_function = "classification"
    model_out.metadata["framework"] = model_framework
    model_out.metadata["type"] = model_type
    model_out.metadata["model function"] = model_function
    model_out.metadata["modified by"] = model_user

    component_outputs = NamedTuple(
        "Outputs",
        [
            ("metrics_thr", float),
        ],
    )

    return component_outputs(float(avg_precision_score))

#### Author your pipeline
Next you will author the pipeline using the KFP SDK. This pipeline consists of the following steps:

* Ingest features
* Create Vertex AI Dataset
* Train XGBoost model
* Evaluate model
* Condition
* Create endpoint
* Deploy model into endpoint

In [ ]:
@dsl.pipeline(
    pipeline_root=PIPELINE_ROOT,
    name=PIPELINE_NAME,
)
def pipeline(
    project_id: str = PROJECT_ID,
    region: str = REGION,
    bucket_name: str = f"gs://{BUCKET_NAME}",
    feature_store_id: str = FEATURESTORE_ID,
    read_instances_uri: str = READ_INSTANCES_URI,
    replica_count: int = 1,
    machine_type: str = "n1-standard-4",
    # train_split: float = 0.8,
    # test_split: float = 0.1,
    # val_split: float = 0.1,
    metrics_uri: str = METRICS_URI,
    thold: float = AVG_PR_THRESHOLD,
):
    # Ingest data from featurestore
    ingest_features_op = ingest_features_gcs(
        project_id=project_id,
        region=region,
        bucket_name=bucket_name,
        feature_store_id=feature_store_id,
        read_instances_uri=read_instances_uri,
    )

    # # create dataset
    dataset_create_op = TabularDatasetCreateOp(
        project=project_id,
        display_name=DATASET_NAME,
        gcs_source=ingest_features_op.outputs["snapshot_uri_paths"],
    ).after(ingest_features_op)

    worker_pool_specs = [
        {
            "machine_spec": {
                "machine_type": "n1-standard-4",
                # Enable if you want to use GPU.
                # "accelerator_type": "NVIDIA_TESLA_T4",
                # "accelerator_count": 1,
            },
            "replica_count": 1,
            "container_spec": {
                "image_uri": IMAGE_URI,
                "args": ARGS,
            },
        }
    ]
    # custom training job component - script
    train_model_op = CustomTrainingJobOp(
        display_name=JOB_NAME,
        worker_pool_specs=worker_pool_specs,
        base_output_directory=bucket_name,
        project=project_id,
        location=region,
    ).after(ingest_features_op)

    importer_spec = dsl.importer(
        artifact_uri=f"gs://{BUCKET_NAME}/model",
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={"containerSpec": {"imageUri": MODEL_SERVING_IMAGE_URI}},
    )
    importer_spec.after(train_model_op)

    model_upload_task = ModelUploadOp(
        project=PROJECT_ID,
        display_name=f"{PIPELINE_NAME}-kfp-model-upload-job",
        unmanaged_container_model=importer_spec.output,
    )

    # evaluate component
    evaluate_model_op = evaluate_model(
        model_in=model_upload_task.outputs["model"],
        metrics_uri=metrics_uri
    ).after(train_model_op)
    
    # if threshold on avg_precision_score
    with dsl.If(
        evaluate_model_op.outputs["metrics_thr"] > thold, name=AVG_PR_CONDITION
    ):
        # create endpoint
        create_endpoint_op = EndpointCreateOp(
            display_name=ENDPOINT_NAME, project=project_id
        ).after(evaluate_model_op)

        # deploy the model
        custom_model_deploy_op = ModelDeployOp(
            model=model_upload_task.outputs["model"],
            endpoint=create_endpoint_op.outputs["endpoint"],
            deployed_model_display_name=MODEL_NAME,
            dedicated_resources_machine_type=machine_type,
            dedicated_resources_min_replica_count=replica_count,
            dedicated_resources_max_replica_count=1,
        ).after(create_endpoint_op)

#### Compile and run the pipeline
After authoring the pipeline you can use the compiler to compile the pipeline. 

In [ ]:
# compile the pipeline
pipeline_compiler = compiler.Compiler()
pipeline_compiler.compile(pipeline_func=pipeline, package_path=PIPELINE_PACKAGE_PATH)

Next you can use the Vertex AI SDK to create a job on Vertex AI Pipelines. 

In [ ]:
# instantiate pipeline representation
pipeline_job = vertex_ai.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path=PIPELINE_PACKAGE_PATH,
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

In [ ]:
# submit the pipeline run (may take ~20 minutes for the first run)
pipeline_job.run(sync=False)

### END

Now you can go to the next notebook `07_deployment.ipynb` and explore deployment using Cloud Build